In [1]:
# A transit-timing example:

In [1]:
# Read in the transit-timing code:
include("../src/ttv.jl")

findtransit3! (generic function with 2 methods)

In [2]:
# Specify the initial conditions:

# Mass of star:
mstar = 0.82

# First planet:
m_1 = 3.18e-4
e1 = 0.0069
om1 = 0.0
p1 = 228.774
t01 = 50.0

# Second planet:
m_2 = 3e-6
e2  = 0.0054
om2 = 0.0
p2 = 221.717
t02 = -p2/6  # We want mean anomaly to be +60 deg, so its
             # transit should have about occurred 1/6 of an orbit 
             # prior to the initial time.

# Now, integration quantities:
t0 = 0.0  # initial time
h  = 10.0 # time step
tmax = 9837.282 # Maximum time of integration

# Okay, set up array for orbital elements
n = 2
elements = zeros(n,7)
elements[1,1] = mstar
elements[2,:] = [m_1,p1,t01,e1*cos(om1),e1*sin(om1),pi/2,0.0]
#elements[3,:] = [m_2,p2,t02,e2*cos(om2),e2*sin(om2),pi/2,0.0]
count = zeros(Int64,n)
tt = zeros(n,44)
rstar = 1e12

# Now, run the ttv function:
dq = ttv_elements!(n,t0,h,tmax,elements,tt,count,0.0,0,0,rstar)

# Print the times:
t1 = tt[2,1:count[2]]
println(t1)

[50.00000000000001, 278.774, 507.548, 736.3219999999999, 965.0959999999999, 1193.8699999999997, 1422.6439999999998, 1651.418, 1880.1919999999998, 2108.966, 2337.74, 2566.514, 2795.288, 3024.062, 3252.8360000000002, 3481.61, 3710.3840000000005, 3939.1580000000004, 4167.932000000001, 4396.706000000001, 4625.4800000000005, 4854.254000000001, 5083.028000000001, 5311.8020000000015, 5540.576000000002, 5769.350000000002, 5998.1240000000025, 6226.898000000002, 6455.672000000002, 6684.446000000003, 6913.220000000003, 7141.994000000003, 7370.768000000003, 7599.542000000003, 7828.316000000003, 8057.090000000004, 8285.864000000003, 8514.638000000004, 8743.412000000004, 8972.186000000005, 9200.960000000005, 9429.734000000004, 9658.508000000005]


In [ ]:
# Plot these times with a mean ephemeris removed:
using Statistics
using PyPlot

# Three different lengths of transit timing sequences:
nplot = [8,22,43]
for iplot=1:2
  pavg = mean(t1[2:nplot[iplot]] - t1[1:nplot[iplot]-1])
  it = collect(0:1:nplot[iplot]-1)
  ttv1 = t1[1:nplot[iplot]] .- it .* pavg .- t1[1]
  if iplot == 2
    plot(it,ttv1,"o")
  else
    plot(it,ttv1,".")
  end
  println(pavg)
end